# Assignment IBM: Segmenting and Clustering Neighborhoods in Toronto, Canada

## Task 1: Extract data from Wikipedia and create Dataframe

### Transform Data

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

In [2]:
# download url data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
data_can = BeautifulSoup(source, 'lxml')

In [3]:
# creat a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
data_to = pd.DataFrame(columns = column_names)

In [4]:
content = data_can.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    data_to = data_to.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood}, ignore_index=True)

# clean dataframe 
data_to = data_to[data_to.Borough!='Not assigned']
data_to = data_to[data_to.Borough!= 0]
data_to.reset_index(drop = True, inplace = True)
i = 0
for i in range(0, data_to.shape[0]):
    if data_to.iloc[i][2] == 'Not assigned':
        data_to.iloc[i][2] = data_to.iloc[i][1]
        i = i+1
                                 
df = data_to.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Data Cleaning

In [6]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [7]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df1 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df1.rename(columns={'Postalcode':'Postal Code'}, inplace=True)

In [9]:
#display the top 10 rows
print(df1.shape)
df1.head(10)

(103, 3)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
print('The DataFrame shape is', df1.shape)

The DataFrame shape is (103, 3)


In [11]:
df1.to_csv('Toronto.Task1.csv',index=False)